In [1]:
# Jupyter Notebook Cell 1: 导入模块
import os
import re
from pathlib import Path

root_dir = Path(".")  # 项目根目录
preview_only = False   # True: 仅打印替换计划；False: 实际执行

# 文件名/目录名转换规则：CamelCase -> snake_case
def to_snake_case(name: str):
    name = name.replace('GPU','_GPU_').replace('__','_')
    name = name.replace('RTE','_RTE_').replace('__','_')
    # name = name.replace('DG_','')
    name = name.replace('DG','_DG_').replace('__','_')
    name = name.replace('InComp','incomp').replace('__','_')
    name = name.replace('inComp','incomp').replace('__','_')
    name = name.replace('_.','.')
    if (name[-1] == '_'): name = name[:-1]
    if (name[0] == '_'): name = name[1:]
    s1 = re.sub("(.)([A-Z][a-z]+)", r"\1_\2", name)
    return re.sub("([a-z0-9])([A-Z])", r"\1_\2", s1).lower().replace('2_d','_2d').replace('__','_')

# 忽略 external、build、.git 等目录
def is_skipped_dir(p: Path):
    return any(part in {"external", "build", "Profiler", ".git", "__pycache__"} for part in p.parts)

# 找出所有 .h/.cpp/.cu/.cuh/.impl.h 等文件
def collect_source_files():
    file_list = []
    for f in root_dir.rglob("*"):
        if f.is_file() and not is_skipped_dir(f):
            if re.search(r"\.(h|cpp|cu|cuh|impl\.h)$", str(f)):
                file_list.append(f)
    return file_list


In [2]:
# Jupyter Notebook Cell 2: 模拟重命名和路径调整
from collections import defaultdict

rename_map = {}           # 原始路径 => 新路径
include_replacements = defaultdict(list)  # 文件 => list of (old_include, new_include)

files = collect_source_files()
for path in files:
    relative = path.relative_to(root_dir)
    parts = list(relative.parts)

    # 转换所有中间目录（跳过 include 和 src 本身）
    for i in range(1, len(parts) - 1):  # skip include/src 根目录
        parts[i] = to_snake_case(parts[i])

    # 转换文件名
    old_name = parts[-1]
    new_name = to_snake_case(old_name)
    parts[-1] = new_name

    new_path = Path(*parts)
    if new_path != relative:
        rename_map[relative] = new_path

# 显示重命名建议
print("📝 重命名建议（模拟）:")
for old, new in rename_map.items():
    print(f"  {old}  →  {new}")


📝 重命名建议（模拟）:


In [7]:
import shutil

# 执行文件移动
for old, new in rename_map.items():
    new.parent.mkdir(parents=True, exist_ok=True)
    shutil.move(str(old), str(new))
print("✅ 文件重命名完成。")

# 修改 include 路径（假设不以 include/ 开头）
def fix_include_lines(text, file_path):
    for old_path, new_path in rename_map.items():
        if old_path.suffix not in {".h", ".impl.h", ".cuh"}:
            continue
        old_inc = old_path.as_posix()[8:] if old_path.as_posix().startswith("include/") else old_path.name
        new_inc = new_path.as_posix()[8:] if new_path.as_posix().startswith("include/") else new_path.name
        text = text.replace(f'#include "{old_inc}"', f'#include "{new_inc}"')
    return text

# 扫描替换
for path in root_dir.rglob("*"):
    if path.is_file() and path.suffix in {".h", ".cpp", ".cu", ".cuh", ".impl.h"}:
        content = path.read_text(encoding="utf-8")
        new_content = fix_include_lines(content, path)
        if new_content != content:
            path.write_text(new_content, encoding="utf-8")
print("✅ include 路径替换完成。")

# 删除空目录（旧结构）
def delete_empty_dirs(root_path):
    for path in sorted(root_path.rglob("*"), reverse=True):
        if path.is_dir() and not any(path.iterdir()):
            path.rmdir()
delete_empty_dirs(root_dir)
print("✅ 旧目录清理完成。")

✅ 文件重命名完成。
✅ include 路径替换完成。
✅ 旧目录清理完成。


In [11]:
import os
import shutil
from pathlib import Path
import re

def camel_to_snake(name):
    """将 CamelCase 或 PascalCase 转换为 snake_case（保留原缩写逻辑）"""
    # print(name,re.sub(r'(?<!^)(?=[A-Z])', '_', name).lower())
    # return re.sub(r'(?<!^)(?=[A-Z])', '_', name).lower()
    name = name.replace('GPU','_GPU_').replace('__','_')
    name = name.replace('RTE','_RTE_').replace('__','_')
    # name = name.replace('DG_','')
    name = name.replace('DG','_DG_').replace('__','_')
    name = name.replace('InComp','incomp').replace('__','_')
    name = name.replace('inComp','incomp').replace('__','_')
    name = name.replace('_.','.')
    if (name[-1] == '_'): name = name[:-1]
    if (name[0] == '_'): name = name[1:]
    s1 = re.sub("(.)([A-Z][a-z]+)", r"\1_\2", name)
    s1 = re.sub("([a-z0-9])([A-Z])", r"\1_\2", s1).lower().replace('2_d','_2d').replace('1_d','_1d').replace('__','_')
    # print(s1)
    return s1

root = Path("./examples")

# 搜索所有目录（旧目录）
old_dirs = [p for p in root.glob("**/") if p.is_dir()]

# 建立可能的映射关系（旧 → 新），例如 Example/Euler2D → example/euler_2d
moved_makefiles = []

for old_dir in old_dirs:
    dir_parts = old_dir.parts
    if not dir_parts:
        continue

    new_parts = tuple(camel_to_snake(part) for part in dir_parts)
    new_dir = Path(*new_parts)

    # 判断 Makefile 是否存在
    makefile = old_dir / "draw.ipynb"
    if makefile.exists() and new_dir.exists() and new_dir != old_dir:
        dst = new_dir / "draw.ipynb"
        shutil.move(str(makefile), str(dst))
        moved_makefiles.append((makefile, dst))

# 打印移动结果
print("✅ 已移动 Makefile 文件:")
for src, dst in moved_makefiles:
    print(f"  {src} → {dst}")


✅ 已移动 Makefile 文件:
  examples/cylinder/cylinder_Re105_50/draw.ipynb → examples/cylinder/cylinder_re105_50/draw.ipynb
  examples/cylinder/cylinder_Re105/draw.ipynb → examples/cylinder/cylinder_re105/draw.ipynb
  examples/cylinder/cylinder_Re105_new/draw.ipynb → examples/cylinder/cylinder_re105_new/draw.ipynb
  examples/RadiationTransfer/Ex1/draw.ipynb → examples/radiation_transfer/ex1/draw.ipynb
  examples/Euler1D/Riemann1D_Toro/draw.ipynb → examples/euler_1d/riemann_1d_toro/draw.ipynb
  examples/Euler1D/Sod_GPU/draw.ipynb → examples/euler_1d/sod_gpu/draw.ipynb
  examples/Euler1D/SineWave1D_GPU/draw.ipynb → examples/euler_1d/sine_wave_1d_gpu/draw.ipynb
  examples/Euler1D/Riemann1D_01007288_GPU/draw.ipynb → examples/euler_1d/riemann_1d_01007288_gpu/draw.ipynb
  examples/Euler1D/Sod1D/draw.ipynb → examples/euler_1d/sod_1d/draw.ipynb
  examples/Euler1D/BlastWave_GPU/draw.ipynb → examples/euler_1d/blast_wave_gpu/draw.ipynb
  examples/Euler1D/Riemann1D_Toro_GPU/draw.ipynb → examples/euler_1d

In [6]:
# Jupyter Notebook Cell 3: 模拟 include 替换建议
for old_path, new_path in rename_map.items():
    if old_path.suffix in {".h", ".impl.h", ".cuh"}:
        old_include = f'#include "{old_path.as_posix()[8:]}"'
        new_include = f'#include "{new_path.as_posix()[8:]}"'
        for src in files:
            if src.suffix in {".h", ".cpp", ".cu"}:
                text = src.read_text(encoding="utf-8")
                if old_include in text:
                    include_replacements[src].append((old_include, new_include))

# 显示替换建议
print("\n🔄 include 替换建议（模拟）:")
for src, changes in include_replacements.items():
    print(f"\n📄 {src}:")
    for old_inc, new_inc in changes:
        print(f"    {old_inc}  →  {new_inc}")



🔄 include 替换建议（模拟）:

📄 src/Runner/RunCompressibleEuler/RunCompressibleEuler_inst_HLL.cu:
    #include "Runner/RunCompressibleEuler/RunCompressibleEuler_impl.h"  →  #include "runner/run_compressible_euler/run_compressible_euler_impl.h"

📄 src/Runner/RunCompressibleEuler/RunCompressibleEuler_inst_HLLEM.cu:
    #include "Runner/RunCompressibleEuler/RunCompressibleEuler_impl.h"  →  #include "runner/run_compressible_euler/run_compressible_euler_impl.h"

📄 src/Runner/RunCompressibleEuler/RunCompressibleEuler_inst_HLLC.cu:
    #include "Runner/RunCompressibleEuler/RunCompressibleEuler_impl.h"  →  #include "runner/run_compressible_euler/run_compressible_euler_impl.h"

📄 src/Runner/RunCompressibleEuler/RunCompressibleEuler_inst_Roe.cu:
    #include "Runner/RunCompressibleEuler/RunCompressibleEuler_impl.h"  →  #include "runner/run_compressible_euler/run_compressible_euler_impl.h"

📄 src/Runner/RunCompressibleEuler/RunCompressibleEuler_inst_LF.cu:
    #include "Runner/RunCompressibleEuler/RunCom

In [ ]:
import os
import re
import shutil
import pathlib
from typing import List, Tuple
import pandas as pd

# ===== 参数设置 =====
simulate = True  # 是否只模拟
root_dirs = ["include", "src"]
target_dirs = ["DG/DG_Schemes", "Mesh/CGALMesh"]
min_file_count_for_subdir = 1  # ≥1 表示即使一个文件也创建子目录
project_root = "."  # 根目录

# ===== 命名转换规则 =====
def to_snake_case(name: str) -> str:
    # 大驼峰转小写下划线
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()

def get_file_group_key(file_name: str) -> str:
    base = re.sub(r'(_impl|_boundarys|_internals|_cells)?\.h.*$', '', file_name)
    return base.lower()

# ===== 扫描所有文件 =====
def collect_files(root_dirs, target_dirs) -> List[Tuple[str, str]]:
    file_list = []
    for r in root_dirs:
        for td in target_dirs:
            base = os.path.join(r, td)
            for dirpath, _, filenames in os.walk(base):
                for f in filenames:
                    if f.endswith(('.h', '.cpp', '.cu')):
                        full_path = os.path.join(dirpath, f)
                        file_list.append((r, full_path))
    return file_list

# ===== 分组 & 重命名建议 =====
def analyze_structure(file_list):
    move_plan = []
    group_dict = {}

    for root_dir, full_path in file_list:
        rel_path = os.path.relpath(full_path, root_dir)
        dir_name, fname = os.path.split(rel_path)
        base = get_file_group_key(fname)
        if base not in group_dict:
            group_dict[base] = []
        group_dict[base].append((root_dir, dir_name, fname, full_path))

    for group, files in group_dict.items():
        # 子目录名为 base（已 snake_case）
        subdir_name = to_snake_case(group)
        if len(files) >= min_file_count_for_subdir:
            for root_dir, old_dir, old_fname, old_path in files:
                new_fname = to_snake_case(old_fname)
                new_dir = os.path.join(root_dir, os.path.dirname(old_dir), subdir_name)
                new_path = os.path.join(new_dir, new_fname)
                move_plan.append({
                    "old_path": old_path,
                    "new_path": new_path,
                    "type": "move+rename" if old_path != new_path else "noop"
                })
        else:
            for root_dir, old_dir, old_fname, old_path in files:
                new_fname = to_snake_case(old_fname)
                new_path = os.path.join(root_dir, old_dir, new_fname)
                move_plan.append({
                    "old_path": old_path,
                    "new_path": new_path,
                    "type": "rename" if old_path != new_path else "noop"
                })

    return pd.DataFrame(move_plan)

# ===== 替换 include 路径 =====
def update_includes(df: pd.DataFrame):
    include_replacements = []
    for _, row in df.iterrows():
        old = os.path.relpath(row['old_path'], project_root).replace("\\", "/")
        new = os.path.relpath(row['new_path'], project_root).replace("\\", "/")
        if old != new:
            include_replacements.append((old, new))
    return include_replacements

def replace_includes_in_file(file_path: str, replacements: List[Tuple[str, str]]):
    with open(file_path, "r", encoding="utf-8") as f:
        content = f.read()
    new_content = content
    for old, new in replacements:
        old_pattern = re.escape(old)
        new_content = re.sub(rf'#include\s+["<]{old_pattern}[">]', f'#include "{new}"', new_content)
    if new_content != content:
        if not simulate:
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(new_content)
        return True
    return False

# ===== 执行文件迁移与替换 =====
def apply_changes(df: pd.DataFrame):
    changed_files = []
    for _, row in df.iterrows():
        old_path = row['old_path']
        new_path = row['new_path']
        if row['type'] == "noop":
            continue
        if not simulate:
            os.makedirs(os.path.dirname(new_path), exist_ok=True)
            shutil.move(old_path, new_path)
        changed_files.append((old_path, new_path))

    # 替换所有代码文件中的 include
    all_code_files = []
    for r, _, files in os.walk("."):
        for f in files:
            if f.endswith((".h", ".cpp", ".cu")):
                all_code_files.append(os.path.join(r, f))

    include_replacements = update_includes(df)
    replaced = []
    for f in all_code_files:
        if replace_includes_in_file(f, include_replacements):
            replaced.append(f)

    return changed_files, replaced

# ===== 运行入口 =====
files = collect_files(root_dirs, target_dirs)
df_plan = analyze_structure(files)
changed, include_updated = apply_changes(df_plan)

# ===== 可视化输出 =====
display(df_plan[df_plan["type"] != "noop"])
print(f"\n总共需修改: {len(df_plan[df_plan['type'] != 'noop'])} 个文件")
print(f"模拟模式: {simulate}")
